In [ ]:
!pip install datasets transformers evaluate sentencepiece accelerate

In [ ]:
import pandas as pd
from datasets import Dataset

train_df = pd.read_excel('/content/result_4_classes_1200.xlsx')
train_df.head()

,Unnamed: 0,ID урока,Дата сообщения,Текст сообщения,swearing,technical_difficulties,difficulty_level,everything_ok
0,0,307751.0,2024-03-06 10:53:03,"winningConditions = [\n [0, 1, 2],\n [3,...",0,0,0,0
1,1,307752.0,2024-03-13 10:12:22,https://diresnode.com/; 2024-03-13T10:12:22.638Z,1,0,0,0
2,2,321813.0,2024-03-02 14:29:01,пока; 2024-03-02T14:29:01.773Z,0,0,0,0
3,3,321813.0,2024-03-02 14:29:07,пока; 2024-03-02T14:29:07.513Z,0,0,0,0
4,4,321813.0,2024-03-02 14:29:08,досвидания; 2024-03-02T14:29:08.331Z,0,0,0,0


In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1011 entries, 0 to 1010
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Unnamed: 0              1011 non-null   int64  
 1   ID урока                978 non-null    float64
 2   Дата сообщения          976 non-null    object 
 3   Текст сообщения         1011 non-null   object 
 4   swearing                1011 non-null   int64  
 5   technical_difficulties  1011 non-null   int64  
 6   difficulty_level        1011 non-null   int64  
 7   everything_ok           1011 non-null   int64  
dtypes: float64(1), int64(5), object(2)
memory usage: 63.3+ KB


In [ ]:
train_df.swearing=train_df.swearing.astype('float')
train_df.technical_difficulties=train_df.technical_difficulties.astype('float')
train_df.difficulty_level=train_df.difficulty_level.astype('float')
train_df.everything_ok=train_df.everything_ok.astype('float')

In [ ]:
test_df = pd.read_excel('/content/test.xlsx')
test_df.swearing=test_df.swearing.astype('float')
test_df.technical_difficulties=test_df.technical_difficulties.astype('float')
test_df.difficulty_level=test_df.difficulty_level.astype('float')
test_df.everything_ok=test_df.everything_ok.astype('float')
test_df.sample(5)

,Unnamed: 0,ID урока,Дата сообщения,Текст сообщения,swearing,technical_difficulties,difficulty_level,everything_ok
24,1425,337224,2024-03-10 09:25:15,33333,0.0,0.0,0.0,0.0
0,1400,337224,2024-03-10 08:45:16,рптврочрплоншвешшыенняеквегашнонрепкаунгшлгонр...,0.0,0.0,0.0,0.0
103,1506,337226,2024-03-24 08:16:51,пелагея блин,0.0,0.0,0.0,0.0
169,1572,337227,2024-03-31 08:02:38,уже всё,0.0,0.0,0.0,0.0
130,1533,337226,2024-03-24 09:09:04,У МИНЯ ДР 31,0.0,0.0,0.0,0.0


In [ ]:
# from sklearn.model_selection import train_test_split

# train_df, test_df = train_test_split(data, test_size=0.2)

print(train_df['swearing'].value_counts())
print(train_df['technical_difficulties'].value_counts())
print(train_df['difficulty_level'].value_counts())
print(train_df['everything_ok'].value_counts())

train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

swearing
0.0    984
1.0     27
Name: count, dtype: int64
technical_difficulties
0.0    942
1.0     69
Name: count, dtype: int64
difficulty_level
0.0    973
1.0     38
Name: count, dtype: int64
everything_ok
0.0    924
1.0     87
Name: count, dtype: int64


In [ ]:
classes = ['swearing','technical_difficulties','difficulty_level', 'everything_ok']
class2id = {class_:id for id, class_ in enumerate(classes)}
id2class = {id:class_ for class_, id in class2id.items()}

In [ ]:
from transformers import AutoTokenizer

model_path = 'cointegrated/rubert-tiny2'
tokenizer = AutoTokenizer.from_pretrained(model_path)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
def preprocess_function(example):
    text = example['Текст сообщения']
    labels = [example[class_] for class_ in classes]
    example = tokenizer(text, truncation=True)
    example['labels'] = labels

    return example

tokenized_train_dataset = train_dataset.map(preprocess_function)
tokenized_test_dataset = test_dataset.map(preprocess_function)

Map:   0%|          | 0/1011 [00:00<?, ? examples/s]

Map:   0%|          | 0/197 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
import evaluate
import numpy as np

clf_metrics = evaluate.combine(["accuracy", "f1", "precision", "recall"])

def sigmoid(x):
   return 1/(1 + np.exp(-x))

def compute_metrics(eval_pred):
   predictions, labels = eval_pred
   predictions = sigmoid(predictions)
   predictions = (predictions > 0.5).astype(int).reshape(-1)
   return clf_metrics.compute(predictions=predictions, references=labels.astype(int).reshape(-1))

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=len(classes),
                                                           id2label=id2class, label2id=class2id,
                                                           problem_type = "multi_label_classification").to('cuda')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
   output_dir="output_checkpoint",
   learning_rate=2e-5,
   per_device_train_batch_size=8,
   per_device_eval_batch_size=8,
   num_train_epochs=100,
   weight_decay=0.01,
   evaluation_strategy="epoch",
   save_strategy='no',
   load_best_model_at_end=False,
   metric_for_best_model='f1'
)

trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_train_dataset,
   eval_dataset=tokenized_test_dataset,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.194014,0.979695,0.000000,0.000000,0.000000
2,No log,0.136756,0.979695,0.000000,0.000000,0.000000
3,No log,0.121761,0.979695,0.000000,0.000000,0.000000
4,0.265300,0.117511,0.979695,0.000000,0.000000,0.000000
5,0.265300,0.111172,0.979695,0.000000,0.000000,0.000000
6,0.265300,0.094738,0.980964,0.117647,1.000000,0.062500
7,0.265300,0.088573,0.982234,0.222222,1.000000,0.125000
8,0.155600,0.085332,0.982234,0.222222,1.000000,0.125000
9,0.155600,0.083840,0.982234,0.222222,1.000000,0.125000
10,0.155600,0.083021,0.982234,0.222222,1.000000,0.125000


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defin

TrainOutput(global_step=12700, training_loss=0.029358562443434724, metrics={'train_runtime': 429.3039, 'train_samples_per_second': 235.498, 'train_steps_per_second': 29.583, 'total_flos': 67218529062384.0, 'train_loss': 0.029358562443434724, 'epoch': 100.0})

In [ ]:
trainer.save_model('save_model')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import shutil
source_path = '/content/save_model'
destination_path = '/content/drive/My Drive/saved/models_tiny_on_4_classes_1200'
shutil.copytree(source_path, destination_path)

'/content/drive/My Drive/saved/models_tiny_on_4_classes_1200'